<a href="https://colab.research.google.com/github/prakHr/sr-python/blob/master/DeepSpeech2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title
import os
from os.path import exists, join, basename, splitext

if not exists('deepspeech-0.6.1-models'):
  !apt-get install -qq sox
  !pip install -q deepspeech-gpu==0.6.1 
  !wget https://github.com/mozilla/DeepSpeech/releases/download/v0.6.1/deepspeech-0.6.1-models.tar.gz
  !tar xvfz deepspeech-0.6.1-models.tar.gz

Selecting previously unselected package libopencore-amrnb0:amd64.
(Reading database ... 144379 files and directories currently installed.)
Preparing to unpack .../0-libopencore-amrnb0_0.1.3-2.1_amd64.deb ...
Unpacking libopencore-amrnb0:amd64 (0.1.3-2.1) ...
Selecting previously unselected package libopencore-amrwb0:amd64.
Preparing to unpack .../1-libopencore-amrwb0_0.1.3-2.1_amd64.deb ...
Unpacking libopencore-amrwb0:amd64 (0.1.3-2.1) ...
Selecting previously unselected package libmagic-mgc.
Preparing to unpack .../2-libmagic-mgc_1%3a5.32-2ubuntu0.4_amd64.deb ...
Unpacking libmagic-mgc (1:5.32-2ubuntu0.4) ...
Selecting previously unselected package libmagic1:amd64.
Preparing to unpack .../3-libmagic1_1%3a5.32-2ubuntu0.4_amd64.deb ...
Unpacking libmagic1:amd64 (1:5.32-2ubuntu0.4) ...
Selecting previously unselected package libsox3:amd64.
Preparing to unpack .../4-libsox3_14.4.2-3ubuntu0.18.04.1_amd64.deb ...
Unpacking libsox3:amd64 (14.4.2-3ubuntu0.18.04.1) ...
Selecting previously un

**Saving all files after uploading them into single folder**

In [2]:
from google.colab import files

uploaded = files.upload()
!mkdir '/content/singleFolder/'
for fn in uploaded.keys():
  !mv '{fn}' '/content/singleFolder/{fn}'

Saving audio.wav to audio.wav
Saving clean-speech.wav to clean-speech.wav
Saving filename.wav to filename.wav


**After uploading into a single folder run model predictions on corresponding files and saving filepath,predictions to output.txt**

In [23]:
paths=[]
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(name=fn,length=len(uploaded[fn])))
  !deepspeech --model deepspeech-0.6.1-models/output_graph.pbmm --lm deepspeech-0.6.1-models/lm.binary --trie deepspeech-0.6.1-models/trie --audio '/content/singleFolder/{fn}' >> '/content/output.txt'
  filepaths="/content/singleFolder/"+fn
  paths.append(filepaths)


User uploaded file "audio.wav" with length 140162 bytes
Loading model from file deepspeech-0.6.1-models/output_graph.pbmm
TensorFlow: v1.14.0-21-ge77504a
DeepSpeech: v0.6.1-0-g3df20fe
2020-07-06 11:33:27.772034: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2020-07-06 11:33:27.773366: I tensorflow/stream_executor/platform/default/dso_loader.cc:42] Successfully opened dynamic library libcuda.so.1
2020-07-06 11:33:27.792697: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1005] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-07-06 11:33:27.793479: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1640] Found device 0 with properties: 
name: Tesla K80 major: 3 minor: 7 memoryClockRate(GHz): 0.8235
pciBusID: 0000:00:04.0
2020-07-06 11:33:27.793521: I tensorflow/stream_executor/platform/default/dl

In [20]:
print(paths)

['/content/singleFolder/audio.wav', '/content/singleFolder/clean-speech.wav', '/content/singleFolder/filename.wav']


In [24]:
my_file = open("/content/output.txt")
string_list = my_file.readlines()
my_file.close()
my_file = open("/content/output.txt","w")
for i,f in enumerate(paths):
  
  my_file.write(f+","+string_list[i])
my_file.close()

In [25]:
def wer(r, h):
    """
    Calculation of WER with Levenshtein distance.
or fn in uploaded.keys():
    Works only for iterables up to 254 elements (uint8).
    O(nm) time ans space complexity.
  
    Parameters
    ----------
    r : list
    h : list

    Returns
    -------
    int

    Examples
    --------
    >>> wer("who is there".split(), "is there".split())
    1
    >>> wer("who is there".split(), "".split())
    3
    >>> wer("".split(), "who is there".split())
    3
    """
    # initialisation
    import numpy
    #print("Here")
    d = numpy.zeros((len(r) + 1) * (len(h) + 1), dtype=numpy.uint8)
    #print("Here")
    d = d.reshape((len(r) + 1, len(h) + 1))
    for i in range(len(r) + 1):
        for j in range(len(h) + 1):
            if i == 0:
                d[0][j] = j
            elif j == 0:
                d[i][0] = i

    # computation
    for i in range(1, len(r) + 1):
        for j in range(1, len(h) + 1):
            if r[i - 1] == h[j - 1]:
                d[i][j] = d[i - 1][j - 1]
            else:
                substitution = d[i - 1][j - 1] + 1
                insertion = d[i][j - 1] + 1
                deletion = d[i - 1][j] + 1
                d[i][j] = min(substitution, insertion, deletion)

    return d[len(r)][len(h)]

In [26]:
f = open("/content/output.txt", "r")
relative_file_path="/content/singleFolder/"
transcriptions={relative_file_path+"audio.wav":"what do you want",
   relative_file_path+"clean-speech.wav":"at this moment the whole soul of the old man seemed centered in his eyes which became bloodshot the veins of the throat swelled his cheeks and temples became purple as though he was struck with epilepsy nothing was wanting to complete this but the utterance of a cry",
   relative_file_path+"filename.wav":"what do you want"}
average_error_rate=0
iter=0
for i,x in enumerate(f):
  iter=iter+1
  print(x)
  y=x.split(",")
  
  word_error_rate=wer(transcriptions[y[0]].split(),y[1].split())
  average_error_rate+=word_error_rate
  print(word_error_rate)  
average_error_rate=average_error_rate/iter
print(average_error_rate)

/content/singleFolder/audio.wav,what do you want

0
/content/singleFolder/clean-speech.wav,at this moment the whole soul of the old man seemed centered in his eyes which became bloodshot the veins of the throat swelled his cheeks and temples became purple as though he was struck with epilepsy nothing was wanting to complete this but the utterance of a cry

0
/content/singleFolder/filename.wav,but he on

4
1.3333333333333333
